# Exploring Periodicity

## Motivation

After comparing time series plots of various aspects of the data, the total unit sales stuck out as particularly interesting due to what appeared to be a stable periodicity. We judged that if this periodic trend of the total unit sales could be exploited and accurately predicted, then we could just focus on predicting the proportions of sales per store-item pair, effectively standardizing the scale of inputs and expected outputs of any subsequent models.

Here was the motivating plot of total unit sales over the last year, in which you can see a striking consistency in the spacing between spikes. 
![Total Last Year](periodicity/total_over_last_year.png)

## Hypotheses and Assumptions

Since the metric for this competition is the Root-Mean-Squared-Error of the *log* of predictions and truth, we felt it a natural choice to divide our prediction strategy into a magnitude component and proportion component. In doing so, we recognize the greater penalty for wrong-magnitude predictions over same-magnitude errors. Given that there are 54 individual stores and 4100 items, we judged it was fair to assume no single store-item could greatly influence the total unit sales. We hypothesized it would be easier to determine the magnitude of our predictions from the total unit sales than from store-item specific training because as the coarsest view of the data, the total unit sales would likely be the least noisy and suffer less from data sparsity. We planned to refine our view of the data successively (i.e. predict per-store unit sales next) if needed to predict magnitudes at a more detailed scale if needed.